## Get all imports

In [ ]:
import os
import random
import numpy as np
import torch
import numpy as np
from torchvision import datasets, transforms
import math
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score

In [ ]:
def set_seed(seed=27):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

# Create a device variable which will be used to shift model and data to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_seed(seed=27)
device

Random seed set as 27


device(type='cuda')

## Download the data and create a pytorch dataset with necessary augmentations

In [ ]:
train_set = datasets.FashionMNIST(root='data/', train=True, transform=transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.286), (0.3202)),
    transforms.Lambda(torch.flatten)
]), download=True)

# Load the validation set
val_set = datasets.FashionMNIST(root='data/', train=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.286), (0.3202)),
    transforms.Lambda(torch.flatten)
]))


print(len(train_set))
print(len(val_set))

100%|██████████| 26421880/26421880 [00:03<00:00, 8777199.67it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 159428.41it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 2654120.28it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 17511984.58it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

60000
10000


In [ ]:
batch_size = 512
dataloader_train = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
dataloader_val = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=0)

## Create a Feed Forward Neural Network, Multi-Layer Perceptron (MLP)

In [ ]:
class FFN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Define the layers
        self.layers = torch.nn.Sequential(*[torch.nn.Linear(784, 512),
                                           torch.nn.Linear(512, 256),
                                           torch.nn.Linear(256, 64),
                                           torch.nn.Linear(64, 10)])
        # Define the corresponding masks
        self.masks = torch.nn.ParameterList([torch.nn.Parameter(torch.Tensor(torch.ones(512, 784)), requires_grad=False),
                                             torch.nn.Parameter(torch.Tensor(torch.ones(256, 512)), requires_grad=False),
                                             torch.nn.Parameter(torch.Tensor(torch.ones(64, 256)), requires_grad=False),
                                             torch.nn.Parameter(torch.Tensor(torch.ones(10, 64)), requires_grad=False)
                                            ])
    
    def forward(self, x):
        # First multiply each layer's weights with its corresponding mask
        with torch.no_grad():
            for i in range(len(self.layers)):
                self.layers[i].weight.copy_(self.layers[i].weight.data * self.masks[i])
        # Now perform forward pass
        for i in range(len(self.layers)):
            if i < len(self.layers)-1:
                x = torch.nn.ReLU()(self.layers[i](x))
            else:
                return self.layers[i](x)
        return x

## Model training and testing routine for each round

In [ ]:
def train_model(epochs, dataloader, device, model, optimizer, loss_function):
    # Set model to training mode in order to unfreeze all layers and allow gradient propagation
    model.train()
    
    # These two lists will be used to store average loss and accuracy for each epoch
    total_loss, acc = list(), list()
    
    # Now write out the training procedure
    for epoch in range(epochs):
        print("Epoch:", epoch+1)
        # Each batch produces a loss, predictions and target
        batch_loss, batch_preds, batch_target = 0, list(), list()
        
        # For each batch, train the model
        for x, y in tqdm(dataloader, total=len(dataloader)):
            # Make sure that data is on the same device as the model
            x, y = x.to(device), y.to(device)
            
            # Remove all previous gradients
            optimizer.zero_grad()
            
            # Get predictions by performing a forward pass
            preds = model.forward(x.float())  
            
            # Calculate error
            loss = loss_function(preds, y.to(torch.int64))
            
            # Calculate all the gradients for each layer
            loss.backward()
            
            # Finall, update the weights
            optimizer.step()
            
            # Save the loss
            batch_loss+= loss.item()
            # Save the predictions and target
            batch_preds.extend(np.argmax(preds.cpu().detach().numpy(), axis=1))
            batch_target.extend(y.cpu().detach().numpy())
        # Calculate average loss
        total_loss.append(batch_loss/len(dataloader))
        # Calculate accuracy for this epoch
        acc.append(accuracy_score(batch_target, batch_preds))
        print("Loss:", total_loss[-1], "\tAcc:", acc[-1])
    return model, total_loss, acc

def test_model(dataloader, device, model):
    # Set model to eval mode in order to freeze all layers so that no parameter gets updated during testing
    model.eval()
    
    # Each batch produces a loss, predictions and target
    batch_preds, batch_target = list(), list()
    
    # For each batch, train the model
    for x, y in tqdm(dataloader, total=len(dataloader)):
        # Make sure that data is on the same device as the model
        x, y = x.to(device), y.to(device)
        
        preds = model.forward(x.float())
        
        # Save the predictions and target
        batch_preds.extend(np.argmax(preds.cpu().detach().numpy(), axis=1))
        batch_target.extend(y.cpu().detach().numpy())
        
    return accuracy_score(batch_target, batch_preds)

## Validation Accuracy before pruning the model 

In [ ]:
# Set the number of epochs to be used
epochs = 10
# Create the model
model = FFN().to(device)
# Define Loss
loss_function = torch.nn.CrossEntropyLoss()
# Define Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

model, _, _ = train_model(epochs, dataloader_train, device, model, optimizer, loss_function)
init_accuracy = test_model(dataloader_val, device, model)*100
print("Final Accuracy of Model before Pruning: ", init_accuracy)

Epoch: 1


  0%|          | 0/118 [00:00<?, ?it/s]

Loss: 0.5779037553880174 	Acc: 0.7866333333333333
Epoch: 2


  0%|          | 0/118 [00:00<?, ?it/s]

Loss: 0.37804761680506044 	Acc: 0.86145
Epoch: 3


...

Loss: 0.24983558601747125 	Acc: 0.9068
Epoch: 10


  0%|          | 0/118 [00:00<?, ?it/s]

Loss: 0.24387589551634709 	Acc: 0.9086666666666666


  0%|          | 0/20 [00:00<?, ?it/s]

Final Accuracy of Model before Pruning:  87.87


# Defining the model for training and pruning


In [ ]:
def pruning(sparsity, rounds, epochs):
  # Create the model
  model = FFN().to(device)
  # Define Loss
  loss_function = torch.nn.CrossEntropyLoss()
  # Define Optimizer
  optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

  # First save the model weights that have been initialized
  init_weights = [model.layers[i].weight.data.to(device) for i in range(len(model.layers))]

  for round_ in range(rounds):
      print("\n\n\nROUND", round_+1, "Started\n----------------------")
      # First train the model for some epochs
      model, _, _ = train_model(epochs, dataloader_train, device, model, optimizer, loss_function)
      if round_ == 0:
          print("Test Accuracy before pruning:", test_model(dataloader_val, device, model))
      else:
          print("Test Accuracy after pruning and retraining:", test_model(dataloader_val, device, model))
      with torch.no_grad():
          # Now prune the model weights
          for i in range(len(model.layers)):
              num_rows = model.layers[i].weight.data.shape[0]
              num_cols = model.layers[i].weight.data.shape[1]
              # Lottery Ticket Style Pruning
              indices = torch.argsort(torch.reshape(torch.abs(model.layers[i].weight.data), (1, num_rows*num_cols)).squeeze())
              # Since we already have the indices to prune, first reset the parameters
              model.layers[i].weight.copy_(init_weights[i])
              # Now prune
              model.masks[i] = torch.reshape(model.masks[i], (1, num_rows*num_cols)).squeeze()
              val = ((sparsity*100)**((round_+1)/rounds))/100
              model.masks[i][indices[:math.ceil(val*num_rows*num_cols)]] = 0
              model.masks[i] = torch.reshape(torch.reshape(model.masks[i], (1, num_rows*num_cols)).squeeze(), (num_rows, num_cols))              
      print("Test Accuracy after pruning but no retraining:", test_model(dataloader_val, device, model))

  # Retrain the final model, check Validation Accuracy and sparsity statistics
  model, _, _ = train_model(epochs, dataloader_train, device, model, optimizer, loss_function)
  finalAccuracy = test_model(dataloader_val, device, model)
  total_sparsity = 0
  total_params = 0
  for i in range(len(model.layers)):
    total_sparsity+= len(torch.where(model.layers[i].weight.data==0)[0])
    total_params+= torch.numel(model.layers[i].weight.data)
  
  print("Final Test Accuracy after pruning: ", finalAccuracy)
  print("Total Parameters without pruning:", total_params)
  print("Parameters remaining after pruning:", total_params-total_sparsity)
  print("Total Sparsity(%):", (total_sparsity/total_params)*100)
  print("Model compression ratio:", total_params/(total_params-total_sparsity))
  
  return finalAccuracy, total_sparsity, total_params 

## Pruning at different sparsities

In [ ]:
epochs = 5
rounds = 5
sparsities = [0.1, 0.2, 0.3, 0.4, 0.5]
accuracies = list()

for sparsity in sparsities:
  print("\n\n\nSparsity: ", sparsity, " Started\n----------------------")
  acc, _, _, = pruning(sparsity, rounds, epochs)
  accuracies.append(acc*100)




Sparsity:  0.1  Started
----------------------



ROUND 1 Started
----------------------
Epoch: 1


  0%|          | 0/118 [00:00<?, ?it/s]

Loss: 0.5856675207614899 	Acc: 0.7831333333333333
Epoch: 2


  0%|          | 0/118 [00:00<?, ?it/s]

Loss: 0.3794019532910848 	Acc: 0.8605666666666667
Epoch: 3


...

Loss: 0.10863820255055266 	Acc: 0.9579333333333333
Epoch: 5


  0%|          | 0/118 [00:00<?, ?it/s]

Loss: 0.09729972011330774 	Acc: 0.9624666666666667


  0%|          | 0/20 [00:00<?, ?it/s]

Final Test Accuracy after pruning:  0.8981
Total Parameters without pruning: 549504
Parameters remaining after pruning: 274752
Total Sparsity(%): 50.0
Model compression ratio: 2.0


## Plot a graph of sparsity vs validation accuracy

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.lineplot(x=[10, 20, 30, 40, 50], y=[init_accuracy]*5, linestyle="--", label="Baseline", color="black")
sns.lineplot(x=[10, 20, 30, 40, 50], y=accuracies, label="LHT")
plt.ylim(80, 90)
plt.xlabel("Sparsity(%)")
plt.ylabel("Validation Accuracy")
sns.despine()

NameError: ignored